In [1]:
from train import *

config_name = "Hybrid"
config_params = {"use_graph": True, "use_esm2": True}
results = {}
model_params = {
                "model_embedding_size": 64,
                "model_attention_heads": 3,
                "model_layers": 2,
                "model_dropout_rate": 0.25,
                "model_top_k_ratio": 0.5,
                "model_top_k_every_n": 1,
                "model_dense_neurons": 32,
                "use_pooling": True
              }

model_params.update(config_params)

num_classes = 10
feature_size = 8
edge_dim = 2


print('Loading state dict...')
model = HybridModel(feature_size, edge_dim, model_params, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
filename = "states/checkpoint_best_Hybrid_model.pth"
model, optimizer, epoch, best_val_acc = load_checkpoint(model, optimizer, filename)

print('Loading data...')
test_data = ProteinDataset('test_partitions', 'test')
test_loader = DataLoader(test_data, batch_size=32)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
       for i, batch in enumerate(tqdm(test_loader, desc="Processing Batches", total=len(test_loader))):
            batch = batch.to(device)
            sequences = batch.sequence
            labels = batch.y
            
            outputs = model(batch.x, batch.edge_attr, batch.edge_index, batch.batch, sequences)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_loss /= len(test_loader)
    test_acc = correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return test_loss, test_acc, precision, recall, f1, conf_matrix


test_loss, test_acc, precision, recall, f1, conf_matrix = evaluate_model(model, test_loader, criterion, device)

results[config_name] = {
            "test_loss": test_loss,
            "test_accuracy": test_acc,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }

print(f"\nResults for {config_name}:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


architecture_names = {
    (1, 10): {"name": "Mainly Alpha: Orthogonal Bundle", "label": 0},
    (1, 20): {"name": "Mainly Alpha: Up-down Bundle", "label": 1},
    (2, 30): {"name": "Mainly Beta: Roll", "label": 2},
    (2, 40): {"name": "Mainly Beta: Beta Barrel", "label": 3},
    (2, 60): {"name": "Mainly Beta: Sandwich", "label": 4},
    (3, 10): {"name": "Alpha Beta: Roll", "label": 5},
    (3, 20): {"name": "Alpha Beta: Alpha-Beta Barrel", "label": 6},
    (3, 30): {"name": "Alpha Beta: 2-Layer Sandwich", "label": 7},
    (3, 40): {"name": "Alpha Beta: 3-Layer(aba) Sandwich", "label": 8},
    (3, 90): {"name": "Alpha Beta: Alpha-Beta Complex", "label": 9}
}
class_names = [str(key) for key in architecture_names.keys()]
plot_confusion_matrix(conf_matrix, class_names, config_name)


/home/gaa220/miniconda3/envs/medbio-challenge/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/gaa220/miniconda3/envs/medbio-challenge/lib/python3.11/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


Loading state dict...
Loading data...
[TEST] Loaded 481 PyG objects from test_partitions


Processing Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [11:58<00:00, 44.91s/it]
/home/gaa220/miniconda3/envs/medbio-challenge/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Results for Hybrid:
Test Loss: 2.1077
Test Accuracy: 0.1954
Precision: 0.3192
Recall: 0.1954
F1 Score: 0.1166


In [2]:
def plot_confusion_matrix(conf_matrix, class_names, config_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{config_name} Confusion Matrix')
    plt.tight_layout()
    plt.savefig(f'images/{config_name}_confusion_matrix.png')
    plt.close()

architecture_names = {
    (1, 10): {"name": "Mainly Alpha: Orthogonal Bundle", "label": 0},
    (1, 20): {"name": "Mainly Alpha: Up-down Bundle", "label": 1},
    (2, 30): {"name": "Mainly Beta: Roll", "label": 2},
    (2, 40): {"name": "Mainly Beta: Beta Barrel", "label": 3},
    (2, 60): {"name": "Mainly Beta: Sandwich", "label": 4},
    (3, 10): {"name": "Alpha Beta: Roll", "label": 5},
    (3, 20): {"name": "Alpha Beta: Alpha-Beta Barrel", "label": 6},
    (3, 30): {"name": "Alpha Beta: 2-Layer Sandwich", "label": 7},
    (3, 40): {"name": "Alpha Beta: 3-Layer(aba) Sandwich", "label": 8},
    (3, 90): {"name": "Alpha Beta: Alpha-Beta Complex", "label": 9}
}
class_names = [str(key) for key in architecture_names.keys()]
plot_confusion_matrix(conf_matrix, class_names, config_name)